## **Reading CSV from ADLS Gen2 in Databricks using SAS**

In [0]:
# Storage details
storage_account_name = "storagenew23"
container_name = "mysource"
sas_token = "?sv=2024-11-04&ss=bt&srt=co&sp=rwdlacuiytfx&se=2025-09-18T12:50:19Z&st=2025-08-18T04:35:19Z&spr=https&sig=j5N2G3oz7YbX4HEm%2B%2FUcm%2BrpBGjlm5Y9R5rxio9YJsI%3D"   

file_name = "employees.csv"

# Path must be in wasbs format without concatenating token directly
file_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/{file_name}"

# Set Spark config with SAS token
spark.conf.set(f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net", sas_token)

# Read the CSV
df = (spark.read
      .format("csv")
      .option("header", "true")
      .option("inferSchema", "true")
      .load(file_path))

df.show(5)


+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03| AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|  MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|  MK_REP|  6000|            - |       201|           20|
+-----------+---

## **Data Overview**

In [0]:
df.printSchema()
df.describe().show()
df.count()


root
 |-- EMPLOYEE_ID: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- PHONE_NUMBER: string (nullable = true)
 |-- HIRE_DATE: string (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- SALARY: integer (nullable = true)
 |-- COMMISSION_PCT: string (nullable = true)
 |-- MANAGER_ID: string (nullable = true)
 |-- DEPARTMENT_ID: integer (nullable = true)

+-------+------------------+----------+---------+------+------------+---------+----------+-----------------+--------------+------------------+-----------------+
|summary|       EMPLOYEE_ID|FIRST_NAME|LAST_NAME| EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|           SALARY|COMMISSION_PCT|        MANAGER_ID|    DEPARTMENT_ID|
+-------+------------------+----------+---------+------+------------+---------+----------+-----------------+--------------+------------------+-----------------+
|  count|                50|        50|       50|  

50

## **Save as Delta Table**

In [0]:
df.write.format("delta").mode("overwrite").saveAsTable("EmployeesDelta")


In [0]:
spark.sql("SELECT * FROM EmployeesDelta LIMIT 5").show()


+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03| AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|  MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|  MK_REP|  6000|            - |       201|           20|
+-----------+---

## **Transformations**

In [0]:
# 1. Select specific columns
df.select("EMPLOYEE_ID", "FIRST_NAME", "LAST_NAME", "SALARY").show(5)

+-----------+----------+---------+------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|SALARY|
+-----------+----------+---------+------+
|        198|    Donald| OConnell|  2600|
|        199|   Douglas|    Grant|  2600|
|        200|  Jennifer|   Whalen|  4400|
|        201|   Michael|Hartstein| 13000|
|        202|       Pat|      Fay|  6000|
+-----------+----------+---------+------+
only showing top 5 rows


In [0]:
# 2. Filter employees with SALARY > 5000
df.filter(df["SALARY"] > 5000).show(5)

+-----------+----------+---------+--------+------------+---------+------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+------+------+--------------+----------+-------------+
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|MK_REP|  6000|            - |       201|           20|
|        203|     Susan|   Mavris| SMAVRIS|515.123.7777|07-JUN-02|HR_REP|  6500|            - |       101|           40|
|        204|   Hermann|     Baer|   HBAER|515.123.8888|07-JUN-02|PR_REP| 10000|            - |       101|           70|
|        205|   Shelley|  Higgins|SHIGGINS|515.123.8080|07-JUN-02|AC_MGR| 12008|            - |       101|          110|
+-----------+----------+--------

In [0]:
#3. Group by DEPARTMENT_ID and average SALARY
df.groupBy("DEPARTMENT_ID").avg("SALARY").show()

+-------------+------------------+
|DEPARTMENT_ID|       avg(SALARY)|
+-------------+------------------+
|           20|            9500.0|
|           40|            6500.0|
|          100| 8601.333333333334|
|           10|            4400.0|
|           50|3721.7391304347825|
|           70|           10000.0|
|           90|19333.333333333332|
|           60|            5760.0|
|          110|           10154.0|
|           30|            4150.0|
+-------------+------------------+



In [0]:
# 4. Order employees by SALARY descending
df.orderBy(df["SALARY"].desc()).show(5)

+-----------+----------+---------+--------+------------+---------+-------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE| JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+-------+------+--------------+----------+-------------+
|        100|    Steven|     King|   SKING|515.123.4567|17-JUN-03|AD_PRES| 24000|            - |        - |           90|
|        102|       Lex|  De Haan| LDEHAAN|515.123.4569|13-JAN-01|  AD_VP| 17000|            - |       100|           90|
|        101|     Neena|  Kochhar|NKOCHHAR|515.123.4568|21-SEP-05|  AD_VP| 17000|            - |       100|           90|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04| MK_MAN| 13000|            - |       100|           20|
|        205|   Shelley|  Higgins|SHIGGINS|515.123.8080|07-JUN-02| AC_MGR| 12008|            - |       101|          110|
+-----------+----------+

In [0]:
# 5. Top 5 highest paid employees
df.select("EMPLOYEE_ID", "FIRST_NAME", "LAST_NAME", "SALARY").orderBy(df["SALARY"].desc()).show(5)

+-----------+----------+---------+------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|SALARY|
+-----------+----------+---------+------+
|        100|    Steven|     King| 24000|
|        102|       Lex|  De Haan| 17000|
|        101|     Neena|  Kochhar| 17000|
|        201|   Michael|Hartstein| 13000|
|        205|   Shelley|  Higgins| 12008|
+-----------+----------+---------+------+
only showing top 5 rows


## **Visualization**

In [0]:
import pandas as pd

# Example: Convert department-wise average salary to pandas
dept_avg_df = df.groupBy("DEPARTMENT_ID").avg("SALARY").toPandas()
dept_avg_df.rename(columns={"avg(SALARY)": "AVG_SALARY"}, inplace=True)

dept_avg_df.head()


,DEPARTMENT_ID,AVG_SALARY
0,20,9500.000000
1,40,6500.000000
2,100,8601.333333
3,10,4400.000000
4,50,3721.739130


## **1. Bar Chart**

In [0]:
#Average salary grouped by Department ID
dept_avg_salary = df.groupBy("DEPARTMENT_ID").avg("SALARY")
display(dept_avg_salary)

DEPARTMENT_ID,avg(SALARY)
20,9500.0
40,6500.0
100,8601.333333333334
10,4400.0
50,3721.7391304347825
70,10000.0
90,19333.333333333332
60,5760.0
110,10154.0
30,4150.0


Databricks visualization. Run in Databricks to view.

## **2. Line Chart**

In [0]:
#Average salary by Hire Date (trend)
salary_trend = df.groupBy("HIRE_DATE").avg("SALARY").orderBy("HIRE_DATE")
display(salary_trend)

HIRE_DATE,avg(SALARY)
01-MAY-03,7900.0
03-JAN-06,9000.0
05-FEB-06,4800.0
06-APR-06,2500.0
06-FEB-08,2200.0
07-DEC-02,11000.0
07-DEC-07,6900.0
07-FEB-07,4200.0
07-JUN-02,9202.0
07-MAR-06,7800.0


Databricks visualization. Run in Databricks to view.

## **3. Pie Chart**

In [0]:
#Count of employees by Job ID
job_count = df.groupBy("JOB_ID").count()
display(job_count)

JOB_ID,count
FI_ACCOUNT,5
MK_MAN,1
IT_PROG,5
FI_MGR,1
AC_ACCOUNT,1
HR_REP,1
PU_CLERK,5
AC_MGR,1
PR_REP,1
ST_MAN,5


Databricks visualization. Run in Databricks to view.